# 敵対的線形バンディット

**参考**

* [Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)

敵対的な線形バンディットは、未知のパラメータベクトルが敵対者によって選択される確率的線形バンディットの一般化みたいなものです。
この2つのトピックは結構似ています。
実際、ここで使う技術は[確率的線形バンディットで使った最適設計のアイデア](CVX_minimum_volume_ellipsoids.ipynb)と[敵対的バンディット](BANDIT_basics.ipynb)のExp3を組み合わせたものです。

## 表記
* 行動の集合：$\mathcal{A} \subset \mathbb{R}^d$
* Lossベクトルの系列：$\theta_1, \ldots, \theta_n \in \mathbb{R}^d$
* 行動：$A_t\in \mathcal{A}$
* Loss：$Y_t = \langle A_t, \theta_t\rangle$

## 目的関数

敵対的線形バンディットの$n$ラウンド目のリグレットは以下で定義されます：

$$
R_n=\mathbb{E}\left[\sum_{t=1}^n Y_t\right]-\min _{a \in \mathcal{A}} \sum_{t=1}^n\left\langle a, \theta_t\right\rangle
$$

これは明らかに通常の敵対的バンディットの一般化ですね。実際、$\mathcal{A}=\{e_1, \dots, e_d\}$ならば通常の敵対的バンディットになります。

## 線形Exp3アルゴリズム

敵対的バンディットのときと同様に、Exponential Weightを定義します。$s$ラウンド目での行動$a$のLossの推定を$\hat{Y}_s(a)$として、

$$\tilde{P}_t(a) \propto \exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s(a)\right)$$

を考えます。これを使って行動をサンプリングするわけですが、$\hat{Y}$の推定を制御するために、適当な$\pi$との混合分布を考えましょう。つまり、
$\sum_{a \in \mathcal{A}} \pi(a)=1$を満たす$\pi: \mathcal{A} \rightarrow[0,1]$に対して、

$$
P_t(a)=(1-\gamma) \tilde{P}_t(a)+\gamma \pi(a)
$$

を考え、$A_t \sim P_t$とします。

Lossを推定する方法を考えましょう。行動$a$に対して報酬が$\langle a_t, \theta_t\rangle$で表されることを仮定していたので、パラメータ$\theta_t \in \mathbb{R}^d$を推定すれば良さそうです。（EXP3ではダイレクトにLossを予測していました）。

以下では最小二乗法で推定することを考えましょう。
つまり、$\langle a_t, \hat{\theta}\rangle \approx \langle a_t, \theta\rangle = y_t$になるように推定すれば良いわけですね。
$A x = y$の最小二乗法の解は$x=(A^\top A)^{-1}A^\top y$であることを踏まえれば、$(A^\top A)^{-1}$のアナロジーとして$R_t \in \mathbb{R}^{d \times d}$を用意し、$\hat{\theta}_t=R_t A_t Y_t$を考え、$\hat{\theta}_t$が今までの履歴について普遍推定になるように$R_t$ を選択 すれば良さそうです。

$\mathbb{E}_t[\cdot] = \mathbb{E}[\cdot | P_t]$とすると、

$$
\mathbb{E}_t\left[\hat{Y}_t\right]=R_t \mathbb{E}_t\left[A_t A_t^{\top}\right] \theta_t=R_t \underbrace{\left(\sum_{a \in \mathcal{A}} P_t(a) a a^{\top}\right)}_{Q_t} \theta_t
$$

よって、$R_t=Q_t^{-1}$とすれば普遍推定が得られます。$Q_t$はnon-singularになるようにしないといけないですが、これは$Q(\pi)=\sum_{a \in \mathcal{A}} \pi(a) a a^{\top}$ がnon-singularになるように選択すればオッケーです。

以上をまとめると、Exp3アルゴリズムは次のように書けます。

1. Input Finite action set $\mathcal{A} \subset \mathbb{R}^d$, learning rate $\eta$, exploration distribution $\pi$, exploration parameter $\gamma$
2. for $t=1,2, \ldots, n$ do
    1. Compute sampling distribution:
    $$
    P_t(a)=\gamma \pi(a)+(1-\gamma) \frac{\exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s(a)\right)}{\sum_{a^{\prime} \in \mathcal{A}} \exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s\left(a^{\prime}\right)\right)} .
    $$
    2. $\quad$ Sample action $A_t \sim P_t$
    3. $\quad$ Observe loss $Y_t=\left\langle A_t, y_t\right\rangle$ and compute loss estimates:
    $$
    \hat{Y}_t=Q_t^{-1} A_t Y_t \quad \text { and } \quad \hat{Y}_t(a)=\left\langle a, \hat{Y}_t\right\rangle
    $$
3. end for

$Q_t$の逆行列を計算する必要があるため、行動空間が有限でないとこれは計算量が無限になってしまいます。

## リグレット解析

このExp3アルゴリズムは次を満たします。

---

$\mathcal{A}$は空でなく、$k=|\mathcal{A}|$ とします。
このとき、任意の$\pi$について、
$$
R_n \leq 2 \sqrt{(2 g(\pi)+d) n \log (k)}
$$
であるようなパラメータが存在します。ここで、$g(\pi)=\max _{a \in \mathcal{A}}\|a\|_{Q^{-1}(\pi)}^2$です。
また、$2 \sqrt{3 d n \log (k)}$であるような$\pi$が存在します。

---

証明しましょう。まずは通常のExp3アルゴリズムと同様にして、推定値の和を考えましょう。

$$
\hat{L}_n(a)=\sum_{t=1}^n \hat{Y}_t(a), \hat{L}_n=\sum_{t=1}^n\left\langle P_t, \hat{Y}_t\right\rangle \text { and } \tilde{L}_n=\sum_{t=1}^n\left\langle\tilde{P}_t, \hat{Y}_t\right\rangle
$$
とします。
ここで、
$$
\tilde{P}_t(a)=\frac{\exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s(a)\right)}{\sum_{a^{\prime} \in \mathcal{A}} \exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s\left(a^{\prime}\right)\right)}
$$
としました。つまり$P_t=(1-\gamma) \tilde{P}_t+\gamma \pi$です。

Exp3アルゴリズムの証明([BANDIT_adversarial.ipynb](BANDIT_adversarial.ipynb))を思い出すと、リグレットは推定値の和で制御されていました。ここでも同様にして、
$
\mathbb{E}[\hat{L}_n(a)]=\mathbb{E}\left[\sum_{t=1}^n \hat{Y}_t(a)\right]
$
であることに注意すれば、
$$
R_n(a)=\mathbb{E}\left[\hat{L}_n-\hat{L}_n(a)\right]
$$
が成り立ちます。
$P_t$の定義に代入すれば

$$
\hat{L}_n=(1-\gamma) \tilde{L}_n+\gamma \sum_{t=1}^n\left\langle\pi, \hat{Y}_t\right\rangle
$$

です。
また、学習率は$\eta \hat{Y}_t(a) \geq-1$を満たすように選ばれているとします。このとき、[BANDIT_adversarial.ipynb](BANDIT_adversarial.ipynb)と同様にして、
$$
\begin{aligned}
\tilde{L}_n & \leq \hat{L}_n(a)+\frac{\log k}{\eta}+\eta \sum_{t=1}^n\left\langle\tilde{P}_t, \hat{Y}_t^2\right\rangle \\
& \leq \hat{L}_n(a)+\frac{\log k}{\eta}+\frac{\eta}{1-\gamma} \sum_{t=1}^n\left\langle P_t, \hat{Y}_t^2\right\rangle
\end{aligned}
$$
が得られます。ここで$\tilde{P}_t=\frac{P_t-\gamma \pi}{1-\gamma} \leq \frac{P_t}{1-\gamma}$を使っています。



これを使うと、
$$
\begin{aligned}
\hat{L}_n-\hat{L}_n(a) &=(1-\gamma) \tilde{L}_n+\gamma \sum_{t=1}^n\left\langle\pi, \hat{Y}_t\right\rangle - \hat{L}_n(a)\\
& \leq \gamma \sum_{t=1}^n\left\langle\pi, \hat{Y}_t\right\rangle+(1-\gamma) \hat{L}_n(a)+\frac{\log k}{\eta}+\eta \sum_{t=1}^n\left\langle P_t, \hat{Y}_t^2\right\rangle-\hat{L}_n(a) \\
& =\frac{\log k}{\eta}+\eta \sum_{t=1}^n\left\langle P_t, \hat{Y}_t^2\right\rangle+\gamma \sum_{t=1}^n\left\langle\pi-e_a, \hat{Y}_t\right\rangle,
\end{aligned}
$$

ここで、$-1 \leq\left\langle a, y_t\right\rangle \leq 1$なので、
$$
\mathbb{E}\left[\left\langle\pi-e_a, \hat{Y}_t\right\rangle\right]=\left\langle\pi-e_a, y_t\right\rangle \leq 2
$$
が成り立ちます。
よって、
$$
R_n \leq \max _a R_n(a) \leq \frac{\log k}{\eta}+2 \gamma n+\eta \sum_{t=1}^n \mathbb{E}\left[\left\langle P_t, \hat{Y}_t^2\right\rangle\right]
$$
が成立します。

続いて$\eta$と$\gamma$を調整しましょう。
ここで、$M_t=\sum_a P_t(a) \hat{Y}_t^2(a)$とします。
すると
$$
\hat{Y}_t^2(a)=\left(a^{\top} Q_t^{-1} A_t Y_t\right)^2=Y_t^2 A_t^{\top} Q_t^{-1} a a^{\top} Q_t^{-1} A_t
$$
なので、$M_t=\sum_a P_t(a) \hat{Y}_t^2(a)=Y_t^2 A_t^{\top} Q_t^{-1} A_t \leq A_t^{\top} Q_t^{-1} A_t \leq \operatorname{trace}\left(A_t A_t^{\top} Q_t^{-1}\right)$であり、
$$
\mathbb{E}\left[M_t \mid P_t\right] \leq \operatorname{trace}\left(\sum_{a \in \mathcal{A}} P_t(a) a a^{\top} Q_t^{-1}\right)=d
$$
が成り立ちます。
$|Y_t| \leq 1$なので、
$$
\left|\eta \hat{Y}_t(a)\right|=\left|\eta a^{\top} Q_t^{-1} A_t Y_t\right| \leq \eta\left|a^{\top} Q_t^{-1} A_t\right|
$$
が成り立ちます。また、$Q(\pi)=\sum_{a \in \mathcal{A}} \pi(a) a a^{\top}$の定義より、$Q_t \succeq \gamma Q(\pi)$です。続いて次の不等式を活用します。

---

**逆行列の不等式**

$A, B \in \mathbb{R}^{d\times d}$として、$A \succeq B$かつ$B$は可逆行列とします。このとき、任意の$x\in \mathbb{R}^d$について

$$
\|x\|_{A^{-1}}^2=\left\langle x, A^{-1} x\right\rangle \leq\|x\|_{B^{-1}}\left\|A^{-1} x\right\|_B \leq\|x\|_{B^{-1}}\left\|A^{-1} x\right\|_A=\|x\|_{B^{-1}}\|x\|_{A^{-1}}
$$

なので、$\|x\|_{A^{-1}} \leq\|x\|_{B^{-1}}$が成り立ちます。
よって$\|x\|_{B^{-1}}^2-\|x\|_{A^{-1}}^2=\|x\|_{B^{-1}-A^{-1}}^2 \geq 0$であり、$B^{-1}\succeq A^{-1}$です。

---

これを使うと$Q_t^{-1} \preceq Q(\pi)^{-1}/\gamma$が成り立ちます。コーシー・シュワルツの不等式より、
$$
\left|a^{\top} Q_t^{-1} A_t\right| \leq\|a\|_{Q_t^{-1}}\left\|A_t\right\|_{Q_t^{-1}} \leq \max _{\nu \in \mathcal{A}} \nu^{\top} Q_t^{-1} \nu \leq \frac{1}{\gamma} \max _{\nu \in \mathcal{A}} \nu^{\top} Q^{-1}(\pi) \nu=\frac{g(\pi)}{\gamma}
$$
です。よって$\left|\eta \hat{Y}_t(a)\right| \leq \frac{\eta}{\gamma} g(\pi)$なので、$\gamma=\eta g(\pi)$ならば$\left|\eta \hat{Y}_t(a)\right| \leq 1$が保証されます。

よって、$\eta=\sqrt{\frac{\log (k)}{(2 g(\pi)+d) n}}$とすれば、
$$
R_n \leq \frac{\log k}{\eta}+\eta n(2 g(\pi)+d)=2 \sqrt{(2 g(\pi)+d) n \log (k)}
$$
が成り立ち、リグレットがバウンドされました。
このバウンドはさらにKiefer-Wolfowitzを使えば$g(\pi)\leq d$でバウンドできます。

## 無限アームの場合

上のリグレットは$\log(k)$に依存していますが、これはアームの数が無限個あると無限に吹き飛んでしまいますね。
これを回避するには

$$\sup _{a \in \mathcal{A}} \min _{b \in \mathcal{C}} \sup _{y \in \mathcal{L}}|\langle a-b, y\rangle| \leq 1 / n$$

を満たすような有限な$\mathcal{C} \subseteq \mathcal{A}$を見つければ良さそうです(Covering Numberの話かも)。
実際、$\log |\mathcal{C}| \leq d \log (6 d n)$を満たすような$C$が存在します（[Bandit本](https://tor-lattimore.com/downloads/book/book.pdf)Exercise 27.6）が、これは$d$に指数的に依存しています。
実は$\mathcal{A}$が凸集合であれば、もっと計算効率の良いアプローチがあります。

まず、
$$
\tilde{P}_t(B)=\frac{\int_B \exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s(a)\right) d a}{\int_{\mathcal{A}} \exp \left(-\eta \sum_{s=1}^{t-1} \hat{Y}_s(a)\right) d a} .
$$
について、$P_t=(1-\gamma) \tilde{P}_t+\gamma \pi$を導入します。

TODO: ここから先の話
